In [12]:
from os.path import splitext, join, exists, isdir,basename,abspath,dirname
from os import makedirs
import trackpy as tp
import nd2reader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pims
import pims_nd2
from glob import glob
%matplotlib inline

In [2]:
@pims.pipeline
def average_z(image):
    return image.mean(axis=0)  # the same as image[0] + ... + image[4]

def plot_msd(imsd,emsd,ax):
    imsd.reset_index().plot(x="lag time [s]",legend=None,alpha=0.75,ax=ax)
    ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
           xlabel='lag time $t$')
    ax.set_xscale('log')
    ax.set_yscale('log')

    emsd.plot(x="lagt",style='o',legend=None,ax=ax)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
           xlabel='lag time $t$')
    plt.tight_layout()
    return ax

def plot_emsd(expt_data,ax_emsd):
    expt_data.plot(x="lagt",style='o',ax=ax_emsd)
    ax_emsd.set_xscale('log')
    ax_emsd.set_yscale('log')
    ax_emsd.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
                xlabel='lag time $t$')
    ax_emsd.legend(loc = 'center left', bbox_to_anchor = (1.0, 0.5)) #bbox_to_anchor=(2.2, 1.0)
    plt.tight_layout()

def nd2msd(nd_fh):
    frames=pims.ND2_Reader(nd_fh)
    if len(np.shape(frames))==4:
        frames = average_z(frames)
    f_batch = tp.batch(frames,diameter=11,threshold=np.percentile(frames,75))

    t = tp.link_df(f_batch, search_range=11, memory=3)
    t_flt = tp.filter_stubs(t, 3*int(len(frames)/4))
    d = tp.compute_drift(t_flt)
    t_cor = tp.subtract_drift(t_flt, d)
    # plt.figure()
    # tp.plot_traj(t_flt)
    # plt.figure()
    # d.plot()
    imsd=tp.imsd(t_cor,0.1,0.2, max_lagtime=100, statistic='msd')
    emsd=tp.emsd(t_cor,0.1,0.2, max_lagtime=100)
    return imsd,emsd

In [3]:
def expt2plots(expt_info,expt_dh):
    if not exists(expt_dh):
        makedirs(expt_dh)
    expt_data=pd.DataFrame()    
    for test in expt_info:
        test_info=expt_info.loc[:,test]
        test_data=pd.DataFrame()    
        for rep in test_info:
            if not pd.isnull(rep[0]):
                repn="%s %s" % (test,rep[0])
                print repn
                nd_fh=rep[1]
                out_fn=splitext(basename(nd_fh))[0]
                out_fh=expt_dh+out_fn
                if not exists(out_fh+".imsd.png"):
                    imsd,emsd=nd2msd(nd_fh)
                    emsd=pd.DataFrame(emsd)
                    emsd.columns=[repn]
                    emsd=emsd.reset_index()
                    imsd.to_csv(out_fh+".imsd")
                    emsd.to_csv(out_fh+".emsd")

                    fig = plt.figure(figsize=(3, 3))
                    ax_imsd=plt.subplot(111)
                    ax_imsd=plot_msd(imsd,emsd,ax_imsd)
                    ax_imsd.figure.savefig(out_fh+".imsd.png");plt.clf();plt.close()
                else:
                    emsd=pd.read_csv(out_fh+".emsd")
                    del emsd["Unnamed: 0"]
                if len(test_data)==0:
                    test_data=emsd
                else:
        #                 test_data[repn]=emsd[repn]
                    test_data=pd.concat([test_data,emsd[repn]],axis=1)
            test_data.to_csv(expt_dh+test+".emsd")
        if len(expt_data)==0:
            expt_data=test_data
        else:
    #             expt_data.loc[:,test_data.columns.tolist()]=test_data.loc[:,test_data.columns.tolist()]
            expt_data=pd.concat([expt_data,test_data.loc[:,[col for col in test_data.columns.tolist() if col != "lagt"]]],axis=1)
#     expt_data=expt_data.drop_duplicates("lagt",keep='first')
    expt_data.to_csv(expt_dh+"expt.emsd")

    fig = plt.figure(figsize=(6, 3))
    ax_emsd=plt.subplot(121)
    ax_emsd=plot_emsd(expt_data,ax_emsd)
    plt.savefig(expt_dh+"emsd.png");plt.clf();plt.close()
    return expt_data

In [59]:
def expt_dh2expt_info(expt_dh):
    expt_info=pd.read_csv(expt_dh+"info")
    cols_del=["dh","fn_lead"]
    for col in expt_info.columns.tolist():
        if "Unnamed" in col:
            cols_del.append(col)
    for col in cols_del:
        if col in expt_info.columns.tolist():
            del expt_info[col]    
    expt_info2=expt_info.drop(["smpn"],axis=1).T
    expt_info2.columns=expt_info.loc[:,"smpn"]

    for col in expt_info2.columns.tolist():
        for i in range(len(expt_info2)):
            if not pd.isnull(expt_info2.loc["replicate %d" % (i+1),col]):
                expt_info2.loc[("replicate %d" % (i+1)),col]=['replicate %d' % (i+1),expt_info2.loc[("replicate %d" % (i+1)),col]]
            else:
                expt_info2.loc[("replicate %d" % (i+1)),col]=[np.nan,np.nan]
    expt_info2=expt_info2.reset_index()
    del expt_info2["index"]
    return expt_info2

In [49]:
def createinfo(expt_dh):
    info=pd.read_csv(expt_dh+"info")
    for i in range(len(info)):
        reps=glob("%s/%s*" % (info.loc[i,"dh"],info.loc[i,"fn_lead"]))
        for repi,rep in enumerate(reps):
            info.loc[i,"replicate %d" % (repi+1)]=rep
    #     break
    info.to_csv(expt_dh+"info")

In [ ]:
ph={"ph 5.5":\
    [["replicate 1","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int5s_z5_001.nd2"],\
     ["replicate 2","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int5s_z5_002.nd2"],\
     ["replicate 3","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int5s_z5_003.nd2"]],\
    "ph 7.0":\
    [["replicate 1","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph7.0_int5s_z5_001.nd2"],\
     ["replicate 2","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph7.0_int5s_z5_002.nd2"],\
     ["replicate 3","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph7.0_int5s_z5_003.nd2"]]} 
expt_info=pd.DataFrame(ph)
expt_dh="../../../data/yeast_uNS_diffussion/data/160511_uNS_pH/"

In [ ]:
pbs_trial={"PBS":\
    [["replicate 1","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160512_uNS_PBS_trial/pbs_int5s_z5_001.nd2"],\
     ["replicate 2","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160512_uNS_PBS_trial/pbs_int5s_z5_002.nd2"],\
     ["replicate 3","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160512_uNS_PBS_trial/pbs_int5s_z5_003.nd2"]]}
expt_info=pd.DataFrame(pbs_trial)
expt_dh="../../../data/yeast_uNS_diffussion/data/160512_uNS_PBS_trial/"

In [ ]:
ph={"ph 5.5":\
    [["replicate 1","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int5s_001.nd2"],\
     ["replicate 2","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int5s_002.nd2"],\
     ["replicate 3","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int5s_003.nd2"],\
     ["replicate 4","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int5s_004.nd2"]],\
    "ph 7.0":\
    [["replicate 1","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph7.0_int5s_001.nd2"],\
     ["replicate 2","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph7.0_int5s_002.nd2"],\
     ["replicate 3","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph7.0_int5s_003.nd2"],\
     [np.nan,np.nan]]} 
expt_info=pd.DataFrame(ph)
expt_dh="../../../data/yeast_uNS_diffussion/data/160511_uNS_pH/int5s_no_zs/"

In [ ]:
ph={"ph 5.5":\
    [["replicate 1","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int1s_001.nd2"],\
     ["replicate 2","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int1s_002.nd2"],\
     ["replicate 3","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int1s_003.nd2"]],\
    "ph 7.0":\
    [["replicate 1","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph7.0_int1s_001.nd2"],\
     ["replicate 2","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph7.0_int1s_002.nd2"],\
     ["replicate 3","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph7.0_int1s_003.nd2"]]} 
expt_info=pd.DataFrame(ph)
expt_dh="../../../data/yeast_uNS_diffussion/data/160511_uNS_pH/int1s_no_zs/"

In [ ]:
ph={"ph 5.5":\
    [["replicate 1","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int1s_001.nd2"],\
     ["replicate 2","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int1s_002.nd2"],\
     ["replicate 3","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph5.5_int1s_003.nd2"]],\
    "ph 7.0":\
    [["replicate 1","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph7.0_int1s_001.nd2"],\
     ["replicate 2","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph7.0_int1s_002.nd2"],\
     ["replicate 3","/media/Transcend/propro/writ/prjs/2_chem_chap_screens/data/yeast_uNS_gfp/160511_uNS_pH/sd_ph7.0_int1s_003.nd2"]]} 
expt_info=pd.DataFrame(ph)
expt_dh="../../../data/yeast_uNS_diffussion/data/160511_uNS_pH/int1s_no_zs/"

In [ ]:
expt_data_ph_trial=expt2plots(expt_info,expt_dh)

In [ ]:
expt_data_pbs_trial=expt2plots(expt_info,expt_dh)

In [ ]:
expt_dh="../../../data/yeast_uNS_diffussion/data/160514_chems_trial/"

In [ ]:
expt_data_chem_trial=expt2plots(expt_dh2expt_info(expt_dh),expt_dh)

In [60]:
#160516
expt_dh="../../../data/yeast_uNS_diffussion/data/160516_chem_trail/"
createinfo(expt_dh)
expt_data_chem_trial=expt2plots(expt_dh2expt_info(expt_dh),expt_dh)

Frame 114: 8 trajectories present
PBS r'$42^{0}C$' replicate 2


KeyError: "PBS r'$42^{0}C$' replicate 2"

In [58]:
# tmp=pd.read_csv(expt_dh+"info")
del tmp['Unnamed: 0.1']

In [ ]:
tmp.iloc[0,1]

In [ ]:
tmp.to_csv("test")

In [ ]:
fig = plt.figure(figsize=(6, 3))
ax_emsd=plt.subplot(121)
ax_emsd=plot_emsd(expt_data_ph_trial,ax_emsd)
plt.savefig(expt_dh+"emsd.png");
plt.clf();plt.close()

In [ ]:
plot_msd(imsd,emsd,ax_imsd)

In [ ]:
expt_dh

In [ ]:
fig = plt.figure(figsize=(3, 3))
ax_emsd=plt.subplot(111)
expt_data.reset_index().plot(x="lagt",style='o',ax=ax_emsd)
ax_emsd.set_xscale('log')
ax_emsd.set_yscale('log')
ax_emsd.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
            xlabel='lag time $t$')
ax_emsd.legend(bbox_to_anchor=(2.2, 1.0))

In [ ]:
expt_data.columns.tolist()

In [ ]:
imsd,emsd=nd2msd(nd_fh)

In [ ]:
imsd.head()

In [ ]:
frames=pims.ND2_Reader(nd_fh)
if len(np.shape(frames))==4:
    frames = average_z(frames)
# frames.

In [ ]:
f = tp.locate(frames[0],diameter=11,threshold=np.percentile(frames[0],75))
plt.figure()  # make a new figure
tp.annotate(f, frames[0])
f.shape

In [ ]:
pd.concat([f,f_batch],axis=1)

In [ ]:
f_batch = tp.batch([frames[0]],diameter=11,threshold=np.percentile([frames[0]],75))
plt.figure()  # make a new figure
tp.annotate(f_batch, frames[0])
f_batch.shape

In [ ]:
hist_data=np.ravel(frames[0])
plt.hist(hist_data, bins=100, range=(0.0, 6000.0))

In [ ]:
features_all = pd.DataFrame()
i=0
for frame in frames:
    features = tp.locate(frame,diameter=11,threshold=np.percentile(frames[0],75))
    frame_no = i
    features['frame'] = frame_no  # just counting iterations
    if len(features_all)==0:
        features_all=features
    else:
        features_all.append(features)
    i+=1